In [1]:
import tensorflow as tf
import numpy as np

C:\Users\haho6\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [220]:
from keras.preprocessing.image import load_img, img_to_array
images = []
for i in range(470):
    image = load_img('./DATA/'+str(i)+'.jpeg',target_size=(64,64))
    image = img_to_array(image)
    image.shape
    image = image / 256
    images.append(image)
    
    
images = np.array(images)

In [3]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-3., 3., size = [batch_size, n_noise])

In [4]:
def windowUp(arr):
    import cv2
    frame = cv2.cvtColor(np.array(arr), cv2.COLOR_BGR2RGB)
    cv2.imshow("test", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [221]:
X = tf.placeholder(tf.float32, [None, 64,64,3])
Y = tf.placeholder(tf.float32, [None, 1])
Z = tf.placeholder(tf.float32, [None, 100])

In [222]:
def generate(Z):
    w1 = tf.Variable(tf.random_normal([100, 16384]))
    h1 = tf.matmul(Z,w1)
    re = tf.reshape(h1,[-1,4,4,1024])
    re = tf.layers.batch_normalization(re)

    conv_h1 = tf.layers.conv2d_transpose(re ,512, 5, 2, padding='same')
    conv_h1 = tf.nn.relu(conv_h1)

    conv_h2 = tf.layers.conv2d_transpose(conv_h1 ,256, 4, 2, padding='same')
    conv_h2 = tf.nn.relu(conv_h2)

    conv_h3 = tf.layers.conv2d_transpose(conv_h2 ,128, 4, 2, padding='same')
    conv_h3 = tf.nn.relu(conv_h3)
    
#    conv_h4 = tf.layers.conv2d_transpose(conv_h3 ,64, 4, 2, padding='same')
#    conv_h4 = tf.nn.relu(conv_h4)
    
    conv_h6 = tf.layers.conv2d_transpose(conv_h3 ,3, 4, 2, padding='same')
    return conv_h6

In [223]:
def discriminaster(inputs):
        W1 = tf.Variable(tf.random_normal([3,3,3,32],stddev=0.02))
        H1 = tf.nn.conv2d(inputs,W1, strides=[1,2,2,1], padding="SAME")
        H1 = tf.layers.batch_normalization(H1)
        H1 = tf.nn.relu(H1)

        W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.02))
        H2 = tf.nn.conv2d(H1,W2, strides=[1,2,2,1], padding="SAME")
        H2 = tf.nn.relu(H2)

        W3 = tf.Variable(tf.random_normal([3,3,64,256],stddev=0.02))
        H3 = tf.nn.conv2d(H2,W3, strides=[1,2,2,1], padding="SAME")
        H3 = tf.nn.relu(H3)

        out = tf.reshape(H3,[-1,16384])
        out = tf.layers.dense(out,1)
        D = [W1, W2, W3]
        return out, D

In [224]:
G = generate(Z)
D_real = discriminaster(X)
D_gene = discriminaster(G)

In [225]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real,labels=tf.ones_like(D_real)))

loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene,labels=tf.zeros_like(D_gene)))

In [243]:
D_loss = tf.reduce_mean(D_real) - tf.reduce_mean(D_gene)
G_loss = -tf.reduce_mean(D_gene)
clip_D = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in theta_D]

NameError: name 'theta_D' is not defined

In [ ]:
D_solver = (tf.train.RMSPropOptimizer(learning_rate=5e-5)
            .minimize(-D_loss, var_list=theta_D))
G_solver = (tf.train.RMSPropOptimizer(learning_rate=5e-5)
            .minimize(G_loss, var_list=theta_G))

In [226]:
loss_D = loss_D_gene + loss_D_real
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [227]:
train_D = tf.train.AdamOptimizer(0.01).minimize(loss_D)
train_G = tf.train.AdamOptimizer(0.01).minimize(loss_G)

In [228]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [241]:
im = images[250:350]

for i in range(200):
    noise = get_noise(1,100)
    _, varsD=sess.run([train_D, loss_D], 
                  feed_dict={
                      X:im,
                      Z: noise})
    
    _, varsG=sess.run([train_G, loss_G], 
                  feed_dict={
                      X:im,
                      Z: noise})
    print(str(i)+"   D : "+str(varsD)+"\t G : "+str(varsG))


0   D : 0.5320679	 G : 0.8907993
1   D : 0.53206587	 G : 0.8908027
2   D : 0.5320638	 G : 0.8908061
3   D : 0.5320618	 G : 0.89080954
4   D : 0.5320598	 G : 0.8908129
5   D : 0.5320578	 G : 0.8908162
6   D : 0.532056	 G : 0.89081943
7   D : 0.53205407	 G : 0.8908227
8   D : 0.53205216	 G : 0.890826
9   D : 0.5320503	 G : 0.8908291
10   D : 0.53204846	 G : 0.8908322
11   D : 0.5320466	 G : 0.8908353
12   D : 0.5320449	 G : 0.8908384
13   D : 0.53204304	 G : 0.8908414
14   D : 0.5320413	 G : 0.89084446
15   D : 0.53203964	 G : 0.8908474
16   D : 0.532038	 G : 0.8908503
17   D : 0.53203624	 G : 0.8908533
18   D : 0.5320346	 G : 0.8908561
19   D : 0.53203297	 G : 0.89085895
20   D : 0.53203136	 G : 0.89086175
21   D : 0.53202975	 G : 0.89086455
22   D : 0.53202814	 G : 0.89086735
23   D : 0.53202665	 G : 0.89087003
24   D : 0.5320251	 G : 0.8908727
25   D : 0.5320236	 G : 0.89087534
26   D : 0.5320221	 G : 0.8908781
27   D : 0.53202057	 G : 0.8908807
28   D : 0.5320192	 G : 0.8908832
29   

In [242]:

noise = get_noise(20,100)
i = sess.run(G, feed_dict={Z:noise})
for row in range(20):
    windowUp(i[row])

In [388]:
i.shape

(1, 64, 64, 3)

In [207]:
noise = get_noise(1,100)